In [30]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import os
import shutil
import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [31]:
ant = 'A1'                                         # antena usada (A1 o A2)    
obs_template = np.loadtxt( './templates/obs_template_' + ant + '.dat' , skiprows=0 , dtype=str , usecols=(0) )  # observación originalmente usadada como template

In [32]:
#-----------------------------------------------
# generamos carpetas para guardar los resultados
#-----------------------------------------------

dirName = "./results_nbins_" + ant            # nombre de la carpeta en que guardaremos los resultados
if not os.path.exists(dirName):
    os.mkdir(dirName)

    
os.chdir(dirName + '/')                       # entramos en la carpeta en la cual guadaremos los resultados
    
if not os.path.exists('./timing_files'):
    os.mkdir('./timing_files')                # creamos la carpeta que contendrá los archivos .tim       
    
os.chdir('..')                                # salimos de la carpeta en la cual guadaremos los resultados
    
    
shutil.copy('./J0437-4715.par', './results_nbins_' + ant + '/timing_files/')     # copiamos el archivo .par a la nueva carpeta para poder usar tempo2

'./results_nbins_A1/timing_files/J0437-4715.par'

In [34]:
#--------------------------------------------------------
# Ahora trabajamos con los .pfds de nbin=1024,512,256,...
#--------------------------------------------------------

for nbin in [32, 64, 128, 256, 512, 1024]:

    folder = './' + ant + '_' + str(nbin) + '/'                # nombre de la carpeta que contiene los .pfds
    
    template = 'template_' + str(nbin) + '_' + ant + '.std.sm' # nombre del template
    
    shutil.copy('./templates/' + template, folder)             # copiamos el template a la carpeta

    
    os.chdir(folder)                            # entramos al directorio que contiene los .pfds
    pfd_files = glob.glob('*pfd')               # leemos los .pfds
    

#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

# primero, generamos un string que contenga todos los archivos .pfd menos el que usamos como template

    pfd_files.remove(str(obs_template))     # sacamos de la lista de observaciones a la que usamos como template

    archivos = ''
    for pfd in pfd_files:
        archivos = archivos + pfd + ' '     # hacemos un string que contenga al resto de las observaciones

# ahora llamamos a pat para generar el archivo .tim

    timing_file = "timing_" + str(nbin) + ".tim"       # nombre del archivo .tim

    subprocess.check_call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos, shell = True)
    
    os.chdir('..')                                     # salimos del directorio que contiene los .pfds
    
    shutil.copy(folder + '/' + timing_file, "./results_nbins_" + ant + '/timing_files/')             # copiamos el archivo .tim a la carpeta
    
    # si las observaciones son de A1, hay que hacer una corrección por la frecuencia

    if ant == 'A1':
        subprocess.call(["sed -i -e 's/1414.8750/1414.4375/g' ./results_nbins_" + ant + "/timing_files/" + timing_file], shell=True)